In [1]:
import pandas as pd
import requests
from datetime import datetime
import time
from functools import lru_cache
from tqdm import tqdm
import os

# --- Set your project directory ---
PROJECT_DIR = "after_api"
os.makedirs(PROJECT_DIR, exist_ok=True)

# --- Load your original dataframe ---
df = pd.read_csv("Tornado.csv")  # Adjust path if needed
# --- Step 1: Create event_datetime ---
def parse_event_datetime(row):
    ym = str(int(row['BEGIN_YEARMONTH']))
    year = int(ym[:4])
    month = int(ym[4:])
    day = int(row['BEGIN_DAY'])
    time_raw = int(row['BEGIN_TIME'])

    hour = time_raw // 100
    minute = time_raw % 100

    if hour == 24:
        hour = 0
        minute = 0
        dt = datetime(year, month, day) + pd.Timedelta(days=1)
        return datetime(dt.year, dt.month, dt.day, hour, minute)

    return datetime(year, month, day, hour, minute)

df['event_datetime'] = df.apply(parse_event_datetime, axis=1)

# --- Step 2: Prepare fields for API ---
df['rounded_lat'] = df['BEGIN_LAT'].round(3)
df['rounded_lon'] = df['BEGIN_LON'].round(3)
df['event_date'] = df['event_datetime'].dt.date
df['event_hour'] = df['event_datetime'].dt.strftime("%Y-%m-%dT%H:00")

# Variables to request, this includes soil moisture, temperature, etc.
variables = [
    "temperature_2m", "dew_point_2m", "relative_humidity_2m",
    "precipitation", "rain", "snowfall", "snow_depth",
    "windspeed_10m", "windspeed_100m", "winddirection_10m", "winddirection_100m",
    "windgusts_10m", "surface_pressure",
    "cloudcover", "cloudcover_low", "cloudcover_mid", "cloudcover_high",
    "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm",
    "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"
]

# Ensure the variables are available by checking API documentation
hourly_vars = ",".join(variables)

# --- API Setup ---
BASE_URL = "https://customer-archive-api.open-meteo.com/v1/archive"  # Dedicated API server for commercial use
API_KEY = "xWPlcPnPJKtpKga2"  # Replace this with your actual API key

# --- Step 3: Request function (no waiting for 429) ---
def simple_request(url, params):
    try:
        resp = requests.get(url, params=params, timeout=10)
        if resp.ok:
            return resp
        else:
            print(f"❌ Error {resp.status_code}: {resp.text}")
            print("Response content:", resp.content)  # Log the full response
            return None
    except requests.exceptions.RequestException as e:
        print(f"⚠ Request failed: {e}")
        return None

# --- Step 4: Fetch weather data ---
@lru_cache(maxsize=None)
def fetch_once(lat, lon, date_str):
    params = {
        
        "apikey": API_KEY,
        "latitude": lat,
        "longitude": lon,
        "start_date": date_str,
        "end_date": date_str,
        "hourly": hourly_vars,
        "timezone": "UTC"
    }
    resp = simple_request(BASE_URL, params)
    if resp:
        return resp.json().get("hourly", {})
    else:
        return {}

# --- Step 5: Process single row ---
def process_row(row):
    lat = row['rounded_lat']
    lon = row['rounded_lon']
    date = row['event_date'].isoformat()
    hr = row['event_hour']

    hourly = fetch_once(lat, lon, date)

    if "time" in hourly and hr in hourly["time"]:
        pos = hourly["time"].index(hr)
        entry = {var: hourly[var][pos] for var in variables if var in hourly}
        entry.update({
            "event_datetime": row["event_datetime"],
            "event_latitude": row["BEGIN_LAT"],
            "event_longitude": row["BEGIN_LON"],
            "time": hr
        })
        return entry
    else:
        print(f"⚠ No data found for {row['event_datetime']} at {hr}")
        return None

# --- Step 6: Checkpointing ---
partial_filepath = os.path.join(PROJECT_DIR, "tor_checkpoint.csv")
all_entries = []

if os.path.exists(partial_filepath):
    print("🧹 Loading previous checkpoint...")
    checkpoint_df = pd.read_csv(partial_filepath)
    processed_datetimes = set(checkpoint_df['event_datetime'])
    all_entries = checkpoint_df.to_dict('records')
else:
    print("🚀 Starting fresh (no checkpoint found)...")
    processed_datetimes = set()

batch_size = 50
pending = []

# --- Step 7: Main Processing Loop ---
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    if str(row['event_datetime']) in processed_datetimes:
        continue  # Already processed

    result = process_row(row)
    if result:
        all_entries.append(result)
        pending.append(result)

    # Save checkpoint after batch
    if (idx + 1) % batch_size == 0 or (idx + 1) == len(df):
        if pending:
            pd.DataFrame(all_entries).to_csv(partial_filepath, index=False)
            print(f"💾 Checkpoint saved after {idx+1} rows")
            pending = []

# --- Step 8: Save final result ---
final_df = pd.DataFrame(all_entries)
final_output_path = os.path.join(PROJECT_DIR, "tornado_var.csv")
final_df.to_csv(final_output_path, index=False)
print(f"✅ Done! {len(final_df)} records saved to '{final_output_path}'")


🚀 Starting fresh (no checkpoint found)...


Processing rows:   0%|                     | 50/15641 [00:34<2:29:04,  1.74it/s]

💾 Checkpoint saved after 50 rows


Processing rows:   1%|▏                   | 100/15641 [01:04<2:35:18,  1.67it/s]

💾 Checkpoint saved after 100 rows


...

💾 Checkpoint saved after 15550 rows


Processing rows: 100%|█████████████████▉| 15600/15641 [3:05:55<00:27,  1.49it/s]

💾 Checkpoint saved after 15600 rows


Processing rows: 100%|██████████████████| 15641/15641 [3:06:17<00:00,  1.40it/s]

💾 Checkpoint saved after 15641 rows
✅ Done! 15627 records saved to 'after_api/tornado_var.csv'
